### Modulo Arithmetic for Euclid Algorithm

- Given any 2 integers A and B, we can write $A = Bx + r$, where x is some integer and r is the remainder of A % B 

- Identity: $(A + B) \mod C = (A \mod C + B \mod C) \mod C$
    - Let's write $A = Cx + r_1$, and $B = Cy + r_2$
    - Then $A \mod C = r_1$, and $B \mod C = r_2$, because the $Cx \mod C = Cy \mod C = 0$
    - Also, $A + B = C(x+y) + r_1 + r_2$
    - So $(A+B) \mod C = (C(x+y) + r_1 + r_2) \mod C = (r_1 + r_2) \mod C = (A \mod C + B \mod C) \mod C$

- Identity: $(A * B) \mod C = (A \mod C * B \mod C) \mod C$
    - Let's write $A = Cx + r_1$, and $B = Cy + r_2$
    - Then $A \mod C = r_1$, and $B \mod C = r_2$, because the $Cx \mod C = Cy \mod C = 0$
    - Also, $A * B = C^2 xy + Cx r_2 + Cy r_1 + r_1 r_2$
    - So $(A*B) \mod C = (C^2 xy + Cx r_2 + Cy r_1 + r_1 r_2) \mod C = (r_1 \cdot r_2) \mod C = (A \mod C \cdot B \mod C) \mod C$


- Euclid algorithm: Let $A, B$ be 2 integers, where $A > B$. Let $\text{gcd}(A,B) = d$ be the greatest common denominator of $A$ and $B$. Then $\text{gcd}(A,B) = \text{gcd}(B, A \mod B)$
    - Since $d$ divides both $A$ and $B$, it must be true that $d$ also divides any linear combination of $A$ and $B$
        - Since $A = ad$ and $B = bd$, then $xad + ybd = d (xa + yb)$ must be divisible by $d$ regardless of the values of $x$ and $y$
    - Let's write $A = xB + r$, where $r = A \mod B$
    - Then $r = A - xB$ is a linear combination of $A$ and $B$, and therefore it is also divisible by $d$
    - Therefore, it must be true that $\text{gcd}(A,B) = \text{gcd}(B, r) = \text{gcd}(B, A \mod B)$

- Euclid Algorithm 2: Let $A, B$ be 2 integers, where $A > B$. Let $\text{gcd}(A,B) = d$ be the greatest common denominator of $A$ and $B$. Then $\text{gcd}(A,B) = \text{gcd}(B, A - B)$
    - Same argument. If $A = xD$ and $B = yD$, then $A - B = (x-y)D$ must be divisible by D



### GCD Queries

- Given array of size $N$, and $Q$ queries of $L$ and $R$, you have to print a GCD of array excluding the elements within the range $L ... R$ of the array

- Idea: Since we want to exclude the range $L ... R$, we can take the GCD of range $0 ... L-1$ and GCD of range $L+1 ... R$, then find the $\text{GCD}(\text{GCD}_L, \text{GCD}_R)$ 
    - To be able to do this, we need an additional data structure(s) that can give us the value of the gcd of some range $[0,L-1]$ and the $[L+1 , R]$
    - Let's implement this by creating 2 arrays
        - Prefix array: where each element $i$ of the array represents the gcd up to and including element $i$
        - Suffix array: where each element $i$ of the array represents the gcd from and including element $i$

    - Then for every query $L, R$, we can simply get the gcd up to L-1 in constant time, and the gcd from R+1 in constant time
    - We incur a cost of creating the prefix and suffix, and this creation is linear time. This is considered scalable, since the number of queries can be extremely large

In [47]:
# gcd(0,0)

In [52]:
input_arr = [2,6,9]

def gcd(a, b):
    '''
    Time complexity: O(log(min(a,b)))
        - Logic: since we are taking a%b at each step in the loop, the remainder is at best a/2 - 1. 
            - e.g. 100 % 51 = 49
        - So at each step, even in the worst case, you are halving the value
        - Since loop terminates when one of the values reaches 1, the log only applies to the smallest value
        - This works because gcd(a,b) = gcd(b, a%b)
            - w.l.o.g, assume a > b, and let gcd(a,b) = d
            - Since `d` divides both `a` and `b`, it must also divide any linear combination of `a` and `b`
                - Let's write `a = bx + r`. Then `r = a - bx`
                - Then the remainder r must be also be divisible by `d`
                - But r = a % b by definition
            - Therefore, gcd(a,b) must also be gcd(b,r)
    '''
    if (a == 0) & (b == 0):
        return None

    try: 
        if a % b == 0:
            return b
    except:
        pass
    
    try:
        if b % a == 0:
            return a
    except:
        pass

    return gcd(b, a % b)

def make_gcd_prefix_suffix(input_arr: list) -> list:
    '''
    Each point i can either be the gcd of the array up to and including i, or up to and excluding i. Both are fine, but you just need to tweak the implementation later when calling the array. Here, we make the suffix and prefix arrays at i inclusive

    Time complexity: O(2*N) = O(N) where N is length of input arr
    Space complexity: O(2*N) = O(N)
    '''
    gcd_prefix = [0] * len(input_arr)
    gcd_prefix[0] = input_arr[0]
    
    for i in range(1, len(gcd_prefix)):
        gcd_prefix[i] = gcd(gcd_prefix[i-1], input_arr[i])

    gcd_suffix = [0] * len(input_arr)
    gcd_suffix[-1] = input_arr[-1]
    
    for i in range(len(gcd_suffix)-2, -1, -1):
        gcd_suffix[i] = gcd(gcd_suffix[i+1], input_arr[i]) 

    return gcd_prefix, gcd_suffix
    
gcd_prefix, gcd_suffix = make_gcd_prefix_suffix(input_arr)

def process_gcd_query(input_arr: list, l: int, r: int) -> int:
    l_remove_index = l-1
    r_remove_index = r-1

    gcd_left = 0 if l_remove_index <= 0 else gcd_prefix[l_remove_index-1]
    gcd_right = 0 if r_remove_index >= (len(input_arr)-1) else gcd_suffix[r_remove_index+1]

    return gcd(gcd_left, gcd_right)

process_gcd_query(input_arr, 2, 3)

2